In [1]:
import pandas as pd
import torch
from src import Utils, LSTM
from skorch.dataset import ValidSplit
from copy import deepcopy
from sklearn.base import clone
from torch import optim, nn
import numpy as np

/tmp/ipykernel_68940/1850983858.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/minicon

In [2]:
MODEL_FOLDER = 'model_lstm/dropout-1'

In [3]:
# torch.use_deterministic_algorithms(True)
# %env CUBLAS_WORKSPACE_CONFIG=:4096:8

In [4]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [5]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [6]:
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [7]:
X_train

0         [USERNAME] Palangga ka man sang mga taga Baco...
1                      Who dafuq is Jose Montemayor Jr.???
2        Di na nakakatuwa yung mukha ni Mar Roxas sa TV...
3                      national elections. | via[USERNAME]
4        Binay will be staring in a movie called "The D...
                               ...                        
22764    "Kala ko wala andito pala si Marcos."*pertaini...
22765    sie ~ [USERNAME]Marcos Magnanakaw Marcos Dikta...
22766                    If Mar is BatMarBinay is Bane-ay.
22767    to my moots im sorry in not sorry for flooding...
22768                                Uunlad tayo kay Binay
Name: text, Length: 22769, dtype: object

In [8]:
y_train

0        0
1        0
2        1
3        0
4        1
        ..
22764    0
22765    1
22766    1
22767    1
22768    0
Name: label, Length: 22769, dtype: int64

In [9]:
X_test

0                          Bakit trending ang Only Binay?
1       Mare @ Cebu [USERNAME][USERNAME] Marcos Never ...
2       Kahit anong gawin ko bakit di ko ma appreciate...
3       Oras na para tayo'y bumoto ng taong mag tataas...
4       VP[USERNAME]is currently in Zamboanga Sibugay ...
                              ...                        
5687      [USERNAME] Laban LeniAngat Buhay LahatLeni Kiko
5688    Nagconcede ka man Maimarwala ka prinnagdala ka...
5689    Did You Know that former Philippine secretary ...
5690           Bakit nakakairita commercial ni Mar Roxas?
5691    To Doc Willie Ong I'd like to believe you are ...
Name: text, Length: 5692, dtype: object

In [10]:
y_test

0       0
1       1
2       1
3       0
4       0
       ..
5687    0
5688    1
5689    0
5690    1
5691    0
Name: label, Length: 5692, dtype: int64

In [11]:
epochs = [100, 200, 300]
learning_rate = [0.1, 0.01, 0.015, 0.02, 0.001, 0.0001, 0.00001]
# learning_rate = [0.02, 0.03, 0.04, 0.05]
batch_size = [16, 32, 64, 128]
hidden_size = [250]
num_layers = [1, 3]
dropout=[0.1, 0.2, 0.5]


In [12]:
for i in range(len(dropout)):
  print(f"Dropout: {dropout[i]}")
  train_lstm = clone(LSTM.LstmPipeline)

  train_lstm.set_params(
    lstm__train_split=None,
    lstm__module__hidden_size=250,
    lstm__module__num_layers=1,
    lstm__module__output_size=2,
    lstm__module__dropout=dropout[i],
    lstm__optimizer__lr=0.015,
    lstm__max_epochs=30,
    lstm__batch_size=32,
    # lstm__optimizer=optim.SGD,
    # lstm__criterion=nn.BCEWithLogitsLoss,
  )

  train_lstm['lstm'].callbacks[0].dirname = f'{MODEL_FOLDER}/{i}/train_lstm'
  train_lstm['lstm'].callbacks[0].monitor = 'train_loss_best'

  train_lstm.fit(X_train, y_train)

  accuracy, recall, precision, f1 = Utils.get_prediction_results(
    X_test,
    y_test,
    train_lstm,
  )

  Utils.save_trained_model(train_lstm, f"{MODEL_FOLDER}/{i}/LSTM")

  history_data_frame = pd.DataFrame(
      train_lstm['lstm'].history
  ).set_index('epoch')
  history_data_frame.to_csv(f'{MODEL_FOLDER}/{i}/lstm_history.csv')

  metrics_data_frame = pd.DataFrame([{
    'accuracy': accuracy,
    'recall': recall,
    'precision': precision,
    'f1': f1,
  }])
  metrics_data_frame.to_csv(f'{MODEL_FOLDER}/{i}/lstm_metrics.csv')

  Utils.seed_random_number_generators()

Dropout: 0.1


/home/nate/miniconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


  0%|          | 0/712 [00:00<?, ?it/s]

  epoch    train_loss    cp     dur
-------  ------------  ----  ------
      1        0.4813     +  5.1726


  0%|          | 0/712 [00:00<?, ?it/s]

...

  0%|          | 0/712 [00:00<?, ?it/s]

     29        0.2843     +  6.3203


  0%|          | 0/712 [00:00<?, ?it/s]

     30        0.2804     +  7.9182
Accuracy: 0.7853127196064652
Recall: 0.7894736842105263
Precision: 0.785639958376691
F1-score: 0.7875521557719055
Ensemble model saved to Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 250, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=250, out_features=2, bias=True)
  ),
))]).pkl
Random number generators seeded.
Dropout: 0.5


KeyboardInterrupt: 

In [ ]:
print("Finished")

Finished
